# What happens:
- Makes the selection and prepares the email
- Then you just make the screenshot and add the contact per Unit 

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/RBUK_Data"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [4]:
%%time
d = pd.read_sql_query("""select distinct concat(sp.book_code,'_SEC') as sec, s.security_reference, s.issuer_code, sp.position_initial_value, nominal
from security s
inner join security_positions sp
on s.security_reference = sp.security_reference
where s.issuer_code like '%DUMMY%' and s.security_family = 'B' and sp.position_initial_value >0""", connection_RAY)

Wall time: 156 ms


### Show selection

In [16]:
d[d.sec.str.contains('RBSPK')]

,sec,security_reference,issuer_code,position_initial_value,nominal
1,RBSPK_B_SEC,AT000B066675,*DUMMY_UNRATED*,2147349.0,2147349.0


### Send the email 

In [7]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = ''
mail.Cc = 'flori.luci@rbinternational.com;'
mail.Subject = ' - Securities with issuer code DUMMY (' + rd + ')'
mail.Body = '''Dear colleagues,

Please your support to update/correct following *Dummy* issuer codes in the SECURITIY table:

Thanks in advance!

KR,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''


mail.Display()